In [1]:
from correctingagent.world.world import *
from correctingagent.agents.teacher import *
from correctingagent.world.rules import Rule
import os
import correctingagent.pddl.ff as ff

from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import GibbsSampling

from correctingagent.models.pgmmodels import PGMModel, PGMPYInference, InferenceType, get_scope, is_overlap, combine_models, SamplingType
from correctingagent.models.prob_model import KDEColourModel
from correctingagent.world import rules
from correctingagent.world.rules import RedOnBlueRule, CorrectionType
from correctingagent.util.CPD_generation import variable_or_CPD, generate_neg_table_cpd, equals_CPD

from collections import namedtuple
import itertools

import networkx as nx
import numpy as np

from pgmpy.factors import factor_product
from pgmpy.inference import Inference
from pgmpy.models import BayesianModel, MarkovChain, MarkovModel
from pgmpy.utils.mathext import sample_discrete
from pgmpy.extern.six.moves import map, range
from pgmpy.sampling import _return_samples

from pgmpy.models.BayesianModel import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from pgmpy.factors.discrete import State

In [6]:
pgm_model = PGMModel(inference_type=InferenceType.BeliefPropagation,
                      max_inference_size=1)

red_cm = KDEColourModel('red')
blue_cm = KDEColourModel('blue')

time = 0
red_on_blue_rules = rules.Rule.generate_red_on_blue_options('red', 'blue')

violations = pgm_model.extend_model(red_on_blue_rules, red_cm, blue_cm, ['b1', 'b2'], time,
                                     correction_type=CorrectionType.TOWER)

pgm_model.observe({'F(b1)': [1, 1, 1], 'F(b2)': [0, 0, 0], f'corr_{time}': 1})


q = pgm_model.query(violations)
# inference = PGMPYInference(pgm_model)
# inference.infer({'F(b1)':[1,1,1], 'F(b2)':[0,0,0], f'corr_{time}':1})
# q = inference.query(violations)
# #
# q = pgm_model.query(violations, [1, 1])
#
# assert(q[violations[0]] - 0.5 < 0.2)
# assert(q[violations[1]] - 0.5 < 0.2)
#
#


time = 1
violations = pgm_model.extend_model(red_on_blue_rules, red_cm, blue_cm, ['b3', 'b4'], time,
                                    correction_type=CorrectionType.TOWER)

pgm_model.observe({'F(b3)': [1, 1, 1], 'F(b4)': [0, 0, 0], f'corr_{time}': 1})

q = pgm_model.query(violations)

assert (q[violations[0]] - 0.5 < 0.2)
assert (q[violations[1]] - 0.5 < 0.2)

print(q)

pgm_model.observe({'red(b3)': 1})

q = pgm_model.query(violations)

assert (q[violations[0]] == 1.0)
assert (q[violations[1]] == 0.0)


fix bw? False
bw 0.5
fix bw? False
bw 0.5
{'V_1(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0.5, 'V_1(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 0.5}


In [9]:
pgm_model.ordered_models[-2:]

In [5]:
red_cm_factor = TabularCPD("red(b1)", 2, [[1, 0]])

In [9]:
print(red_cm_factor)

+-----------+---+
| red(b1)_0 | 1 |
+-----------+---+
| red(b1)_1 | 0 |
+-----------+---+


In [10]:
red_cm_factor.values.shape

(2,)

In [5]:
q2 = q[q._weight > 0]
q2.sum(axis=0)/len(q2)

all y.(blue(y) -> exists x. (red(x) & on(x,y)))         0.576271
all x.(red(x) -> exists y. (blue(y) & on(x,y)))         0.508475
blue(b2)                                                0.559322
red(b1)                                                 0.440678
V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))    0.559322
V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))    0.440678
corr_0                                                  1.000000
_weight                                                 1.000000
dtype: float64

In [3]:
q[violations[0]]

0.485

In [4]:
q[violations[1]]

0.485

In [3]:
sampler = BayesianModelSampling(model)

In [12]:
# sampler.forward_sample(size=100, return_type="dataframe")
sampler.likelihood_weighted_sample(evidence=[State(var="corr_0", state=1)], size=100)
s = sampler.rejection_sample(evidence=[State(var="corr_0", state=1)], size=100)

In [15]:
s

,red(b1),"all x.(red(x) -> exists y. (blue(y) & on(x,y)))",blue(b2),"V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))","all y.(blue(y) -> exists x. (red(x) & on(x,y)))","V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))",corr_0
0,1,1,0,1,0,0,1
1,1,1,0,1,0,0,1
2,1,1,0,1,0,0,1
3,0,0,1,0,1,1,1
4,1,1,0,1,1,0,1
5,0,0,1,0,1,1,1
6,1,1,0,1,0,0,1
7,1,1,0,1,1,0,1
8,0,0,1,0,1,1,1
9,1,1,0,1,0,0,1


In [16]:
sum(s["red(b1)"])/len(s["red(b1)"])
sum(s["blue(b2)"])/100


0.5

In [20]:
(s.sum(axis=0)/100)['red(b1)']

0.5

In [5]:

student = BayesianModel([('diff', 'grade'), ('intel', 'grade')])
cpd_d = TabularCPD('diff', 2, [[0.6], [0.4]])
cpd_i = TabularCPD('intel', 2, [[0.7], [0.3]])
cpd_g = TabularCPD('grade', 3, [[0.3, 0.05, 0.9, 0.5], [0.4, 0.25,
               0.08, 0.3], [0.3, 0.7, 0.02, 0.2]],
               ['intel', 'diff'], [2, 2])
student.add_cpds(cpd_d, cpd_i, cpd_g)
inference = BayesianModelSampling(student)
inference.forward_sample(size=2, return_type='recarray')


[('intel', 'int'), ('diff', 'int'), ('grade', 'int')]


rec.array([(0, 0, 2), (0, 0, 1)],
          dtype=[('intel', '<i8'), ('diff', '<i8'), ('grade', '<i8')])

In [12]:
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.sampling import GibbsSampling
from pgmpy.models import MarkovModel, FactorGraph
model = FactorGraph([('A', 'B'), ('C', 'B')])
factor_ab = DiscreteFactor(['A', 'B'], [2, 2], [1, 2, 3, 4])
factor_cb = DiscreteFactor(['C', 'B'], [2, 2], [5, 6, 7, 8])
model.add_factors(factor_ab, factor_cb)

gibbs = GibbsSampling(model)
gen = gibbs.generate_sample(size=2)
[sample for sample in gen]

[[], []]

In [21]:
model = FactorGraph([('A', 'B'), ('C', 'B')])
factor_ab = DiscreteFactor(['A', 'B'], [2, 2], [1, 2, 3, 4])
factor_cb = DiscreteFactor(['C', 'B'], [2, 2], [5, 6, 7, 8])
model.add_factors(factor_ab, factor_cb)


In [ ]:
model.

In [5]:
pgm_model.models[1].factors

[<DiscreteFactor representing phi(corr_1:2, V_1(all x.(green(x) -> exists y. (orange(y) & on(x,y)))):2, V_1(all y.(orange(y) -> exists x. (green(x) & on(x,y)))):2) at 0x7ffa24559ad0>,
 <DiscreteFactor representing phi(all x.(green(x) -> exists y. (orange(y) & on(x,y))):2) at 0x7ffa24542c10>,
 <DiscreteFactor representing phi(V_1(all x.(green(x) -> exists y. (orange(y) & on(x,y)))):2, green(b2):2, orange(b4):2, all x.(green(x) -> exists y. (orange(y) & on(x,y))):2) at 0x7ffa24542990>,
 <DiscreteFactor representing phi(all y.(orange(y) -> exists x. (green(x) & on(x,y))):2) at 0x7ffa24542290>,
 <DiscreteFactor representing phi(V_1(all y.(orange(y) -> exists x. (green(x) & on(x,y)))):2, green(b2):2, orange(b4):2, all y.(orange(y) -> exists x. (green(x) & on(x,y))):2) at 0x7ffa244e3f50>]

In [5]:
pgm_model.models[1].factors

[<DiscreteFactor representing phi(V_1(all y.(orange(y) -> exists x. (green(x) & on(x,y)))):2, green(b3):2, orange(b4):2, all y.(orange(y) -> exists x. (green(x) & on(x,y))):2) at 0x7ff4eae59a50>,
 <DiscreteFactor representing phi(all x.(green(x) -> exists y. (orange(y) & on(x,y))):2) at 0x7ff4eadf6a90>,
 <DiscreteFactor representing phi(V_1(all x.(green(x) -> exists y. (orange(y) & on(x,y)))):2, green(b3):2, orange(b4):2, all x.(green(x) -> exists y. (orange(y) & on(x,y))):2) at 0x7ff4eae598d0>,
 <DiscreteFactor representing phi(V_3(all x.(green(x) -> exists y. (orange(y) & on(x,y)))):2, green(b3):2, orange(b5):2, all x.(green(x) -> exists y. (orange(y) & on(x,y))):2) at 0x7ff4eadf6c10>,
 <DiscreteFactor representing phi(corr_3:2, V_3(all x.(green(x) -> exists y. (orange(y) & on(x,y)))):2, V_3(all y.(orange(y) -> exists x. (green(x) & on(x,y)))):2) at 0x7ff4eaddfd50>,
 <DiscreteFactor representing phi(corr_1:2, V_1(all x.(green(x) -> exists y. (orange(y) & on(x,y)))):2, V_1(all y.(oran

In [3]:
q

{'green(b3)': 1, 'orange(b4)': 0.0, 'red(b1)': None, 'blue(b2)': None}

In [5]:
pgm_model = PGMModel(inference_type=InferenceType.BeliefPropagation)

red_cm = KDEColourModel('red')
blue_cm = KDEColourModel('blue')

time = 0
red_on_blue_rules = rules.Rule.generate_red_on_blue_options('red', 'blue')

violations = pgm_model.extend_model(red_on_blue_rules, red_cm, blue_cm, ['b1', 'b2'], time, correction_type=CorrectionType.TOWER)

pgm_model.observe({'F(b1)':[1,1,1], 'F(b2)':[0,0,0], f'corr_{time}':1})


q = pgm_model.query(violations)
# # inference = PGMPYInference(pgm_model)
# # inference.infer({'F(b1)':[1,1,1], 'F(b2)':[0,0,0], f'corr_{time}':1})
# # q = inference.query(violations)
# # #
# # q = pgm_model.query(violations, [1, 1])

assert(q[violations[0]] == 0.5)
assert(q[violations[1]] == 0.5)

pgm_model.observe({'red(b1)': 1})

q = pgm_model.query(violations)

assert(q[violations[0]] == 1.0)
assert(q[violations[1]] == 0.0)

fix bw? False
bw 0.5
fix bw? False
bw 0.5
variable_nodes
V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))
corr_0
all y.(blue(y) -> exists x. (red(x) & on(x,y)))
blue(b2)
all x.(red(x) -> exists y. (blue(y) & on(x,y)))
V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))
red(b1)

factor nodes
phi(corr_0:2, V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y)))):2)
phi(blue(b2):2)
phi(V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y)))):2, red(b1):2, blue(b2):2, all y.(blue(y) -> exists x. (red(x) & on(x,y))):2)
phi(V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, red(b1):2, blue(b2):2, all x.(red(x) -> exists y. (blue(y) & on(x,y))):2)
phi(all y.(blue(y) -> exists x. (red(x) & on(x,y))):2)
phi(red(b1):2)
phi(all x.(red(x) -> exists y. (blue(y) & on(x,y))):2)
factor nodes of model
phi(red(b1):2)
phi(V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y)))):2, red(b1):2, blue(b2):2, all y.(blue(y) -> exists x. (red(x) & on(x,y))):

In [7]:
set(pgm_model.models[0].factors)

{<pgmpy.factors.continuous.ContinuousFactor.ContinuousFactor at 0x7feba5f04d90>,
 <DiscreteFactor representing phi(V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, red(b1):2, blue(b2):2, all x.(red(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7feba5f16250>,
 <DiscreteFactor representing phi(V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y)))):2, red(b1):2, blue(b2):2, all y.(blue(y) -> exists x. (red(x) & on(x,y))):2) at 0x7feba5f04ed0>,
 <DiscreteFactor representing phi(all x.(red(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7feba5f04fd0>,
 <DiscreteFactor representing phi(all y.(blue(y) -> exists x. (red(x) & on(x,y))):2) at 0x7feba5f16090>,
 <DiscreteFactor representing phi(corr_0:2, V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y)))):2) at 0x7feba5f16b50>}

In [3]:
for model in pgm_model.models:
    print(model.nodes)
    print(get_scope(model))
    

['F(b1)', 'red(b1)', <pgmpy.factors.continuous.ContinuousFactor.ContinuousFactor object at 0x7f14214a4950>]
{'red(b1)', 'F(b1)'}
['F(b2)', 'blue(b2)', <pgmpy.factors.continuous.ContinuousFactor.ContinuousFactor object at 0x7f14214a4910>]
{'F(b2)', 'blue(b2)'}
[all x.(red(x) -> exists y. (blue(y) & on(x,y))), <DiscreteFactor representing phi(all x.(red(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7f14214b7090>]
{all x.(red(x) -> exists y. (blue(y) & on(x,y)))}
[all y.(blue(y) -> exists x. (red(x) & on(x,y))), <DiscreteFactor representing phi(all y.(blue(y) -> exists x. (red(x) & on(x,y))):2) at 0x7f14214b7650>]
{all y.(blue(y) -> exists x. (red(x) & on(x,y)))}
['V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))', all x.(red(x) -> exists y. (blue(y) & on(x,y))), <DiscreteFactor representing phi(V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, red(b1):2, blue(b2):2, all x.(red(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7f1421490610>, 'red(b1)', 'blue(b2)']
{'red(b1)', 'V_0(all

In [15]:
idx1 = 0
idx2 = 1
print(get_scope(pgm_model.models[idx1]))
print(get_scope(pgm_model.models[idx2]))

print(is_overlap(get_scope(pgm_model.models[idx1]), get_scope(pgm_model.models[idx2])))


model3 = combine_models(pgm_model.models[idx1], pgm_model.models[idx2])

model1 = pgm_model.models[idx1]
model2 = pgm_model.models[idx2]

pgm_model.models.remove(model1)
pgm_model.models.remove(model2)
pgm_model.models.append(model3)
# get_scope(combine_models(pgm_model.models[2], pgm_model.models[3]))
# get_scope(pgm_model.models[3])

{all y.(blue(y) -> exists x. (red(x) & on(x,y)))}
{'F(b2)', 'F(b1)', all y.(blue(y) -> exists x. (red(x) & on(x,y))), 'corr_0', 'blue(b2)', all x.(red(x) -> exists y. (blue(y) & on(x,y))), 'red(b1)', 'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))', 'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))'}
True


In [16]:
for model in pgm_model.models:
    print(model.nodes)
    print(get_scope(model))

[all y.(blue(y) -> exists x. (red(x) & on(x,y))), <DiscreteFactor representing phi(all y.(blue(y) -> exists x. (red(x) & on(x,y))):2) at 0x7f14214b7650>, all x.(red(x) -> exists y. (blue(y) & on(x,y))), <DiscreteFactor representing phi(all x.(red(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7f14214b7090>, 'F(b2)', 'blue(b2)', <pgmpy.factors.continuous.ContinuousFactor.ContinuousFactor object at 0x7f14214a4910>, 'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))', 'red(b1)', <DiscreteFactor representing phi(V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, red(b1):2, blue(b2):2, all x.(red(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7f1421490610>, 'corr_0', 'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))', <DiscreteFactor representing phi(corr_0:2, V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y)))):2, V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y)))):2) at 0x7f1421442f90>, 'F(b1)', <pgmpy.factors.continuous.ContinuousFactor.ContinuousFactor object at 0x7f14214a4950>

In [11]:
pgm_model = PGMModel()

red_cm = KDEColourModel('red')
blue_cm = KDEColourModel('blue')

time = 0
red_on_blue_rules = rules.Rule.generate_red_on_blue_options('red', 'blue')

violations = pgm_model.extend_model(red_on_blue_rules, red_cm, blue_cm, ['b1', 'b2'], time, correction_type=CorrectionType.TOWER)



inference = PGMPYInference(pgm_model.model)
new_model = inference.infer({}, {'F(b1)':[1,1,1], 'F(b2)':[0,0,0], f'corr_{time}':1})
q = inference.query(violations)
#
# q = pgm_model.query(violations, [1, 1])

assert(q[violations[0]] == 0.5)
assert(q[violations[1]] == 0.5)


fix bw? False
bw 0.5
fix bw? False
bw 0.5


In [7]:
f = q[violations[0]]

In [13]:
f.values[0]

0.5

In [3]:
inference.belief_propagation.map_query(variables=['V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))'], evidence=inference.evidence)

{'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0}

In [3]:
model = inference.new_model

AttributeError: 'PGMPYInference' object has no attribute 'new_model'

In [5]:
new_model

In [27]:
pgm_model.observe({'F(b1)':[1,1,1], 'F(b2)':[0,0,0], f'corr_{time}':1})
pgm_model.infer()

/home/mappelgren/Desktop/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:621: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]
/home/mappelgren/Desktop/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:633: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


In [13]:
pgm_model.query(variables=violations)

{'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0.5,
 'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 0.5}

In [14]:
inference.evidence

{'corr_0': 1}

In [5]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel
from pgmpy.inference import BeliefPropagation
bayesian_model = BayesianModel([('A', 'J'), ('R', 'J'), ('J', 'Q'),
                                 ('J', 'L'), ('G', 'L')])
cpd_a = TabularCPD('A', 2, [[0.2], [0.8]])
cpd_r = TabularCPD('R', 2, [[0.4], [0.6]])
cpd_j = TabularCPD('J', 2,
                    [[0.9, 0.6, 0.7, 0.1],
                     [0.1, 0.4, 0.3, 0.9]],
                    ['R', 'A'], [2, 2])
cpd_q = TabularCPD('Q', 2,
                    [[0.9, 0.2],
                     [0.1, 0.8]],
                    ['J'], [2])
cpd_l = TabularCPD('L', 2,
                    [[0.9, 0.45, 0.8, 0.1],
                     [0.1, 0.55, 0.2, 0.9]],
                    ['G', 'J'], [2, 2])
cpd_g = TabularCPD('G', 2, [[0.6], [0.4]])
bayesian_model.add_cpds(cpd_a, cpd_r, cpd_j, cpd_q, cpd_l, cpd_g)
belief_propagation = BeliefPropagation(bayesian_model)
bp = belief_propagation.map_query(variables=['J', 'Q'],
                              evidence={'A': 0, 'R': 0, 'G': 0, 'L': 1})


0

In [3]:
a = {1:2}

In [4]:
a.update({2:3})

In [5]:
a

{1: 2, 2: 3}